# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 12:27PM,247391,19,ACG-2102491364,ACG North America LLC,Released
1,Sep 27 2022 12:27PM,247391,19,ACG-2102491365,ACG North America LLC,Released
2,Sep 27 2022 11:59AM,247388,10,MSP214336,"Methapharm, Inc.",Released
3,Sep 27 2022 11:59AM,247388,10,MSP214337,"Methapharm, Inc.",Released
4,Sep 27 2022 11:47AM,247386,20,8576048,"ACI Healthcare USA, Inc.",Released
5,Sep 27 2022 11:25AM,247385,20,8574771,"ACI Healthcare USA, Inc.",Released
6,Sep 27 2022 11:25AM,247385,20,8574772,"ACI Healthcare USA, Inc.",Released
7,Sep 27 2022 11:25AM,247385,20,8574773,"ACI Healthcare USA, Inc.",Released
8,Sep 27 2022 11:25AM,247385,20,8574774,"ACI Healthcare USA, Inc.",Released
9,Sep 27 2022 11:25AM,247385,20,8574775,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,247381,Released,4
25,247385,Released,23
26,247386,Released,1
27,247388,Released,2
28,247391,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247381,NaN,NaN,4.0
247385,NaN,NaN,23.0
247386,NaN,NaN,1.0
247388,NaN,NaN,2.0
247391,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0.0,0.0,1.0
247205,0.0,1.0,0.0
247293,0.0,0.0,1.0
247294,0.0,0.0,1.0
247308,0.0,15.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247202,0,0,1
247205,0,1,0
247293,0,0,1
247294,0,0,1
247308,0,15,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,0,0,1
1,247205,0,1,0
2,247293,0,0,1
3,247294,0,0,1
4,247308,0,15,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247202,,,1
1,247205,,1,
2,247293,,,1
3,247294,,,1
4,247308,,15,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC
2,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc."
4,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc."
5,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc."
28,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation
63,Sep 27 2022 11:09AM,247381,10,"Methapharm, Inc."
67,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.
74,Sep 27 2022 10:40AM,247372,19,ACG North America LLC
75,Sep 27 2022 10:39AM,247368,10,Bio-PRF
88,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC,,,2
1,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",,,2
2,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",,,1
3,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc.",,,23
4,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation,,,35
5,Sep 27 2022 11:09AM,247381,10,"Methapharm, Inc.",,,4
6,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,,5,2
7,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,,1,
8,Sep 27 2022 10:39AM,247368,10,Bio-PRF,,9,4
9,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC,2,,
1,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",2,,
2,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",1,,
3,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc.",23,,
4,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation,35,,
5,Sep 27 2022 11:09AM,247381,10,"Methapharm, Inc.",4,,
6,Sep 27 2022 10:42AM,247370,10,Eywa Pharma Inc.,2,5,
7,Sep 27 2022 10:40AM,247372,19,ACG North America LLC,,1,
8,Sep 27 2022 10:39AM,247368,10,Bio-PRF,4,9,
9,Sep 27 2022 10:39AM,247371,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC,2,,
1,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",2,,
2,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",1,,
3,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc.",23,,
4,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation,35,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC,2.0,NaN,NaN
1,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
3,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc.",23.0,NaN,NaN
4,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation,35.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 12:27PM,247391,19,ACG North America LLC,2.0,0.0,0.0
1,Sep 27 2022 11:59AM,247388,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Sep 27 2022 11:47AM,247386,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
3,Sep 27 2022 11:25AM,247385,20,"ACI Healthcare USA, Inc.",23.0,0.0,0.0
4,Sep 27 2022 11:14AM,247380,10,ISDIN Corporation,35.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3462999,118.0,14.0,1.0
15,247352,43.0,24.0,0.0
19,989404,27.0,16.0,0.0
20,742064,25.0,0.0,0.0
21,494407,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3462999,118.0,14.0,1.0
1,15,247352,43.0,24.0,0.0
2,19,989404,27.0,16.0,0.0
3,20,742064,25.0,0.0,0.0
4,21,494407,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,118.0,14.0,1.0
1,15,43.0,24.0,0.0
2,19,27.0,16.0,0.0
3,20,25.0,0.0,0.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,118.0
1,15,Released,43.0
2,19,Released,27.0
3,20,Released,25.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,14.0,24.0,16.0,0.0,1.0
Released,118.0,43.0,27.0,25.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,14.0,24.0,16.0,0.0,1.0
2,Released,118.0,43.0,27.0,25.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,14.0,24.0,16.0,0.0,1.0
2,Released,118.0,43.0,27.0,25.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()